In [3]:
!pip install kaggle --quiet


[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip


In [89]:
import pandas as pd
import numpy as np
import glob

In [37]:
glob.glob("*.csv")

['train.csv', 'sample_submission.csv', 'test.csv']

In [38]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

In [39]:
train.head()

,Unnamed: 0,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,...,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender,Id
0,0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,...,none,own,2,0,1,2.349340e+09,yes,skilled employee,male,0
1,1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,...,none,own,1,1,1,NaN,yes,skilled employee,female,1
2,3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,...,none,for free,1,0,2,NaN,yes,skilled employee,male,3
3,5,NaN,36,repaid,education,9055,NaN,2 years,2,single,...,none,for free,1,0,2,2.345788e+09,yes,unskilled resident,male,5
4,7,169.0,36,repaid,car (used),6948,57.0,2 years,2,single,...,none,rent,1,0,1,2.349962e+09,yes,mangement self-employed,male,7


In [40]:
train.columns

Index(['Unnamed: 0', 'checking_balance', 'months_loan_duration',
       'credit_history', 'purpose', 'amount', 'savings_balance',
       'employment_length', 'installment_rate', 'personal_status',
       'other_debtors', 'residence_history', 'property', 'age',
       'installment_plan', 'housing', 'existing_credits', 'default',
       'dependents', 'telephone', 'foreign_worker', 'job', 'gender', 'Id'],
      dtype='object')

In [41]:
modes = train.mode().iloc[0]

In [42]:
def proc_data(df):
    df.fillna(modes, inplace=True)
    df['credit_history'] = pd.Categorical(df.credit_history)
    df['purpose'] = pd.Categorical(df.purpose)
    df['personal_status'] = pd.Categorical(df.personal_status)
    df['other_debtors'] = pd.Categorical(df.other_debtors)
    df['property'] = pd.Categorical(df.property)
    df['installment_plan'] = pd.Categorical(df.installment_plan)
    df['installment_rate'] = pd.Categorical(df.installment_rate)
    df['housing'] = pd.Categorical(df.housing)
    df['foreign_worker'] = pd.Categorical(df.foreign_worker)
    df['job'] = pd.Categorical(df.job)
    df['gender'] = pd.Categorical(df.gender)
    df['existing_credits'] = pd.Categorical(df.existing_credits)
    df['dependents'] = pd.Categorical(df.dependents)

In [43]:
proc_data(train)
proc_data(test)

In [44]:
def convert_str_years(string):
    if 'year' in string:
        return int(string.split()[0])
    else:
        return round(int(string.split()[0]) / 12, 2)

In [45]:
train['employment_length'] = train['employment_length'].apply(convert_str_years)
train['residence_history'] = train['residence_history'].apply(convert_str_years)

In [46]:
cats = ['credit_history', 'purpose', 'personal_status', 'other_debtors', 'property', 'installment_plan', 'installment_rate',\
       'housing', 'foreign_worker', 'job', 'gender', 'existing_credits', 'dependents']
conts = ['employment_length', 'residence_history', 'checking_balance', 'months_loan_duration','amount', 'savings_balance', \
        'age']

dep = 'default'

In [47]:
df = train.copy()

In [48]:
t_id = df['Id']

In [49]:
df = df.drop(['Unnamed: 0', 'telephone', 'Id'], axis=1)

In [50]:
from numpy import random
from sklearn.model_selection import train_test_split

random.seed(42)
trn_df, val_df = train_test_split(df, test_size=0.25)
trn_df[cats] = trn_df[cats].apply(lambda x: x.cat.codes)
val_df[cats] = val_df[cats].apply(lambda x: x.cat.codes)

In [51]:
def xs_y(df):
    xs = df[cats+conts].copy()
    return xs, df[dep] if dep in df else None

trn_xs, trn_y = xs_y(trn_df)
val_xs, val_y = xs_y(val_df)

In [52]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

m = DecisionTreeClassifier(max_leaf_nodes=4).fit(trn_xs, trn_y)

In [53]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(val_y, m.predict(val_xs))

0.327683615819209

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

rf = RandomForestClassifier(100, min_samples_leaf=2)
rf.fit(trn_xs, trn_y)
mean_absolute_error(val_y, rf.predict(val_xs))

0.24293785310734464

In [57]:
test['employment_length'] = test['employment_length'].apply(convert_str_years)
test['residence_history'] = test['residence_history'].apply(convert_str_years)

In [58]:
test_id = test['Id']

In [59]:
test = test.drop(['Unnamed: 0', 'telephone', 'Id'], axis=1)

In [81]:
test[cats] = test[cats].apply(lambda x: x.cat.codes)

In [82]:
random.seed(42)

test_xs, _ = xs_y(test)

In [86]:
preds = rf.predict(test_xs)

In [87]:
prediction = pd.DataFrame(test_id).join(pd.DataFrame(preds, columns=['Predicted']))

In [90]:
prediction.to_csv('submission.csv', index=False)

In [4]:
!kaggle competitions list --sort-by 'earliestDeadline'

Traceback (most recent call last):
  File "/home/codespace/.python/current/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/python/3.10.4/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /home/codespace/.kaggle. Or use the environment method.
